In [1]:
import time, multiprocessing

In [3]:
# 윈도우의 multiprocessing 동작 안함 

def count_down(x):
    while True:
        if x == 0:
            break
            
        print('카운트: %d' %x)
        x -= 1
        time.sleep(1)
        
p = multiprocessing.Process(target=count_down, args=(5, ))

print(p)

p.start()

<Process name='Process-2' parent=12860 initial>


Critical Section이란?
- 여러 프로세스나 스레드가 동시다발적으로 접근할 수 있는 메모리공간(변수 및 객체)

동기화 메커니즘 (Mutex, Semaphore, Spinlock)을 사용하는 이유?
- 데이터 무결성을 보장하기 위해서

In [4]:
import threading

In [6]:
def print_hello():
    print('Hello')
    time.sleep(1)
    print('Finish sleep')
    
if __name__ == '__main__':
    # 현재 시각을 계측
    start = time.perf_counter()
    threads = []
    
    # 5개의 스레드 객체를 생성
    for _ in range(5):
        t = threading.Thread(target = print_hello)
        # start(): 스레드를 구동 가능하게 함
        t.start()
        threads.append(t)
     
    # 생성한 스레드를 모두 구동(Hello 출력)
    # 이후 1초 쉬고 Finish sleep 출력
    for thread in threads:
        # join(): 실제 스레드 구동
        thread.join()
      
    # 현재 시각을 계측
    finish = time.perf_counter()
    
    # 끝 - 시작: 동작하는데 걸린 시간
    print('Finished: ', str(round(finish - start, 2)) , '초')

Hello
Hello
Hello
HelloHello

Finish sleep
Finish sleepFinish sleep
Finish sleep

Finish sleep
Finished:  1.03 초


In [15]:
# Critical Section에서 데이터 무결성이 지켜지지 않아 문제 발생
# +1을 100만번 하는 동작을 2번 -> 200만이 나와야 함
# 하지만 200만이 나오지 않음 -> 데이터 무결성이 지켜지지 않음
# 여기서 Cretical Section은 x 

x = 0

def increment_global():
    global x
    x += 1

def thread_task():
    # _는 아무것도 받지 않을 경우 사용
    for _ in range(1000000):
        increment_global()
        
def thread_main():
    global x
    x = 0
    
    t1 = threading.Thread(target = thread_task)
    t2 = threading.Thread(target = thread_task)    
    
    t1.start()
    t2.start()    
    
    t1.join()
    t2.join()    
    
for i in range(10):
    thread_main()
    print('{0}번째 반복 이후 x = {1} '.format(i, x))

0번째 반복 이후 x = 2000000 
1번째 반복 이후 x = 1851560 
2번째 반복 이후 x = 1729778 
3번째 반복 이후 x = 1685447 
4번째 반복 이후 x = 1878981 
5번째 반복 이후 x = 1685695 
6번째 반복 이후 x = 1655094 
7번째 반복 이후 x = 1813746 
8번째 반복 이후 x = 1663969 
9번째 반복 이후 x = 1917162 


In [16]:
# Cretical Section에 동기화 메커니즘을 적용하여 데이터 무결성을 보장 
# 데이터는 쓰기(write)할 때 무결성이 깨지므로 그 부분에 동기화 적용해야 함

lock = threading.Lock()

x = 0

def increment_global():
    global x
    x += 1

def thread_task():
    for _ in range(1000000):
        lock.acquire()
        increment_global()
        lock.release()
        
def thread_main():
    global x
    x = 0
    
    t1 = threading.Thread(target = thread_task)
    t2 = threading.Thread(target = thread_task)    
    
    t1.start()
    t2.start()    
    
    t1.join()
    t2.join()    
    
for i in range(10):
    thread_main()
    print('{0}번째 반복 이후 x = {1} '.format(i, x))

0번째 반복 이후 x = 2000000 
1번째 반복 이후 x = 2000000 
2번째 반복 이후 x = 2000000 
3번째 반복 이후 x = 2000000 
4번째 반복 이후 x = 2000000 
5번째 반복 이후 x = 2000000 
6번째 반복 이후 x = 2000000 
7번째 반복 이후 x = 2000000 
8번째 반복 이후 x = 2000000 
9번째 반복 이후 x = 2000000 


In [17]:
# 동기화 메커니즘은 Thread에 Context Switching을 유발
# Memory Hierarchy(메모리 계층구조) 관점에서 메모리에 자주 접근하는 것은 좋지 않음
# 동기화 메커니즘을 어디에 적용하느냐에 따라 속도가 달라짐

lock = threading.Lock()

x = 0

def increment_global():
    global x
    x += 1

def thread_task():
    lock.acquire()
    
    for _ in range(1000000):
        increment_global()
        
    lock.release()    
    
def thread_main():
    global x
    x = 0
    
    t1 = threading.Thread(target = thread_task)
    t2 = threading.Thread(target = thread_task)    
    
    t1.start()
    t2.start()    
    
    t1.join()
    t2.join()    
    
for i in range(10):
    thread_main()
    print('{0}번째 반복 이후 x = {1} '.format(i, x))

0번째 반복 이후 x = 2000000 
1번째 반복 이후 x = 2000000 
2번째 반복 이후 x = 2000000 
3번째 반복 이후 x = 2000000 
4번째 반복 이후 x = 2000000 
5번째 반복 이후 x = 2000000 
6번째 반복 이후 x = 2000000 
7번째 반복 이후 x = 2000000 
8번째 반복 이후 x = 2000000 
9번째 반복 이후 x = 2000000 
